### Map (typed) implementation in WebAssembly

#### Usman Asad, Faisal Jaffer, Aleem Haq. Modified in April 2021

In [5]:
def map_proto():
    return """
    ;; create a array
    (func $arr (param $len i32) (result i32)
        (local $offset i32)                              ;; offset
        (set_local $offset (i32.load (i32.const 0)))     ;; load offset from the first i32

        (i32.store (get_local $offset)                   ;; load the length
                   (get_local $len)
        ) 

        (i32.store (i32.const 0)                         ;; store offset of available space                   
                   (i32.add 
                       (i32.add
                           (get_local $offset)
                           (i32.mul 
                               (get_local $len) 
                               (i32.const 4)
                           )
                       )
                       (i32.const 4)                     ;; the first i32 is the length
                   )
        )
        (get_local $offset)                              ;; (return) the beginning offset of the array.
    )
    ;; return the array size
    (func $len_array (param $arr i32) (result i32)
        (i32.load (get_local $arr))
    )
    ;; convert an element index to the offset of memory
    (func $offset_array (param $arr i32) (param $i i32) (result i32)
        (i32.add
             (i32.add (get_local $arr) (i32.const 4))    ;; The first i32 is the array length 
             (i32.mul (i32.const 4) (get_local $i))      ;; one i32 is 4 bytes
        )
    )
    ;; set a value at the index 
    (func $set_array (param $arr i32) (param $i i32) (param $value i32)
        (i32.store 
            (call $offset_array (get_local $arr) (get_local $i)) 
            (get_local $value)
        )  
    )
    ;; get a value at the index 
    (func $get_array (param $arr i32) (param $i i32) (result i32)
        (i32.load 
            (call $offset_array (get_local $arr) (get_local $i)) 
        )
    )
    (func $mod (param $x i32) (param $y i32) (result i32)
        (local $q i32)
        (local $r i32)
        i32.const 0
        local.set $q
        local.get $x
        local.set $r
        loop $label0
          local.get $r
          local.get $y
          i32.ge_s
          if
            local.get $r
            local.get $y
            i32.sub
            local.set $r
            local.get $q
            i32.const 1
            i32.add
            local.set $q
            br $label0
          end
        end
        local.get $r
      )
     (func $div (param $x i32) (param $y i32) (result i32)
        (local $q i32)
        (local $r i32)
        i32.const 0
        local.set $q
        local.get $x
        local.set $r
        loop $label0
          local.get $r
          local.get $y
          i32.ge_s
          if
            local.get $r
            local.get $y
            i32.sub
            local.set $r
            local.get $q
            i32.const 1
            i32.add
            local.set $q
            br $label0
          end
        end
        local.get $q
      )
    (func $hash1 (param $len i32) (param $key i32) (result i32)
     (call $mod (get_local $key) (get_local $len))
    )
    (func $hash2 (param $len i32) (param $key i32) (result i32)
     (call $mod (call $div (i32.add (i32.const 4564)(get_local $key))(get_local $len)) (get_local $len))
    )
 
    (func $node (param $key i32) (param $value i32) (result i32)
        (local $offset i32)                              ;; offset
        (set_local $offset (i32.load (i32.const 0)))     ;; load offset from the first i32

        (i32.store (i32.const 0)                         ;; store offset of available space                   
                   (i32.add
                           (get_local $offset)
                           (i32.const 8)
                       )  
        )
        (i32.store (get_local $offset)   ;; store key
                   (get_local $key)
        ) 
     
        (i32.store (i32.add (get_local $offset) (i32.const 4))    ;; store value
                    (get_local $value)
        ) 
        
        (get_local $offset)                              ;; (return) the beginning offset of the array.
    )
 
    (func $expand_map (param $map i32)
        (local $array1_adr i32)
        (local $array2_adr i32)
        (local $index i32)
        (set_local $array1_adr (i32.load (i32.add (get_local $map) (i32.const 4))))
        (set_local $array2_adr (i32.load (i32.add (get_local $map) (i32.const 8))))
     
        (i32.store (i32.add (get_local $map) (i32.const 4))    
                   (call $arr (i32.mul (i32.load (get_local $array1_adr)) (i32.const 2))))  ;; new array1 of 2x old size
        (i32.store (i32.add (get_local $map) (i32.const 8))    
                   (call $arr (i32.mul (i32.load (get_local $array2_adr)) (i32.const 2))))  ;; new array2 of 2x old size
        (i32.store (get_local $map) (i32.mul (i32.load(get_local $map)) (i32.const 2)))
        (i32.store (i32.add (get_local $map) (i32.const 12)) 
                    (i32.const 0))
        (set_local $index (i32.const 0))
        loop $LOOP1
            (i32.lt_s (get_local $index) (i32.load (get_local $array1_adr)))   
            if
                (if (i32.ne (call $get_array (get_local $array1_adr) (get_local $index)) (i32.const 0)) 
                    (then
                        (call $insert_map (get_local $map) (i32.load (call $get_array (get_local $array1_adr) (get_local $index))) 
                         (i32.load (i32.add (call $get_array (get_local $array1_adr) (get_local $index)) (i32.const 4))))
                    )
                )
                (set_local $index                       
                    (i32.add (get_local $index) (i32.const 1)))
                br $LOOP1
            end
        end
        (set_local $index (i32.const 0))
        loop $LOOP2
            (i32.lt_s (get_local $index) (i32.load (get_local $array2_adr)))   
            if
                (if (i32.ne (call $get_array (get_local $array2_adr) (get_local $index)) (i32.const 0)) 
                    (then
                        (call $insert_map (get_local $map) (i32.load (call $get_array (get_local $array2_adr) (get_local $index))) 
                         (i32.load (i32.add (call $get_array (get_local $array2_adr) (get_local $index)) (i32.const 4))))
                    )
                )
                (set_local $index                       
                    (i32.add (get_local $index) (i32.const 1)))
                br $LOOP2
            end
        end
    )
 
    (func $map (result i32)
        (local $offset i32)                              ;; offset
        (i32.store (global.get $_memsize) (i32.const 4)) 
        (set_local $offset (i32.load (global.get $_memsize)))     ;; load offset from the first i32

        (i32.store (get_local $offset)                   ;; initial map size is 8
                   (i32.const 8)
        ) 

        (i32.store (i32.const 0)                         ;; store offset of available space                   
                   (i32.add 
                       (i32.add
                           (get_local $offset)
                           (i32.const 12)
                       )
                       (i32.const 4)                     ;; the first i32 is the size
                   )
        )
        (i32.store (i32.add (get_local $offset) (i32.const 4))    ;; Array1
                   (call $arr (i32.const 4))
        ) 
     
        (i32.store (i32.add (get_local $offset) (i32.const 8))    ;; Array2
                    (call $arr (i32.const 4))
        ) 
        (i32.store (i32.add (get_local $offset) (i32.const 12))    ;; number of initial recrods is 0
                    (i32.const 0)
        )
        
        (get_local $offset)                              ;; (return) the beginning offset of the array.
    )
 
  
    (func $in_map (param $map i32) (param $key i32) (result i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $node i32)
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key))
           )
            (then
                (return (i32.const 1))
            )
       )
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key))
           )
            (then
                (return (i32.const 1))
            )
       )
       (i32.const 0)
    )
 
    (func $insert_map (param $map i32) (param $key i32) (param $value i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $map_size i32)
       (local $node i32)
       (local $existing i32)
       (local $table i32)
       (local $max_replace i32)
       (local $temp i32)
       (local $numOfRecords i32)
       (if (i32.eq (call $in_map (get_local $map) (get_local $key)) (i32.const 1))
           (then 
            (call $update_key (get_local $map) (get_local $key) (get_local $value))
           )
        (else 
       (set_local $numOfRecords (i32.load(i32.add (get_local $map) (i32.const 12))))
       (set_local $map_size (call $len_map (get_local $map)))
       (if (i32.ge_s 
                    (get_local $numOfRecords) 
                    (call $div (get_local $map_size) (i32.const 2)) ;; if insert into table 1
                ) 
                    (then
                     (call $expand_map (get_local $map))
                    )
       )
       (set_local $max_replace (i32.const 0))
       (set_local $table (i32.const 2))
       (set_local $node (call $node (get_local $key) (get_local $value)))
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
       (set_local $existing (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index))) ;; set existing to node being pushed out

       (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index) (get_local $node)) ;; insert new node into first array
       (i32.store (i32.add (get_local $map) (i32.const 12)) 
                    (i32.add (i32.load(i32.add (get_local $map) (i32.const 12))) (i32.const 1))) ;; numOfRecords + 1
       loop $LOOP
            (i32.and 
             (i32.lt_s (get_local $max_replace) (i32.const 15)) ;; if max replacements is under 5
             (i32.ne (get_local $existing) (i32.const 0)) ;; if exisitng points to a node (not equal to 0)
            )   
            if      
                (if (i32.eq 
                    (get_local $table) 
                    (i32.const 1) ;; if insert into table 1
                ) 
                    (then
                        (set_local $key (i32.load (get_local $existing))) ;; get the key of the existing node
                        (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key))) ;; hash key and find index in array 1
                        (set_local $temp (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index))) ;; set temp to the value in the index
                        (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index) (get_local $existing)) ;; store the exisitng in the index
                        (set_local $existing (get_local $temp)) ;; set exisitng to temp (what was in the index before)
                        (set_local $table (i32.const 2)) ;; set table to 2
                    )
                    (else ;; if insert into table 2
                       (set_local $key (i32.load (get_local $existing)))
                       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
                       (set_local $temp (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
                        (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index) (get_local $existing))
                        (set_local $existing (get_local $temp))
                        (set_local $table (i32.const 1)) ;; set table to 1
                    )
                )
                (set_local $max_replace                       
                    (i32.add (get_local $max_replace) (i32.const 1))) ;; max_replace + 1
                br $LOOP
            end
        end
        (if (i32.eq 
                    (get_local $existing) 
                    (i32.const 0) ;; if nothing in index
                ) 
            (then
                (call $set_array (
           i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index) (get_local $node))
            )
        )
        (if (i32.and (i32.ne (get_local $existing) (i32.const 0)) (i32.eq (get_local $max_replace) (i32.const 15)))
            (then
                (call $expand_map (get_local $map))
                (call $insert_map (get_local $map) (i32.load (get_local $existing)) 
                     (i32.load (i32.add (get_local $existing) (i32.const 4))))
                
            )
        )
            
        ))
       
    )
 
    (func $update_key (param $map i32) (param $key i32) (param $value i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $node i32)
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key))
           )
            (then
                (i32.store (i32.add (get_local $node) (i32.const 4)) (get_local $value))
            )
       )
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key))
           )
            (then
                (i32.store (i32.add (get_local $node) (i32.const 4)) (get_local $value))
            )
       )
    )
 
    (func $get_map (param $map i32) (param $key i32) (result i32)
       (local $arr1_index i32)
       (local $arr2_index i32)
       (local $node i32)
       (local $map_size i32)
       (set_local $map_size (call $len_map (get_local $map)))
       (set_local $arr1_index (call $hash1 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 4)))) (get_local $key)))
       (set_local $arr2_index (call $hash2 (call $len_array (i32.load(i32.add (get_local $map) (i32.const 8)))) (get_local $key)))

       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 4))) (get_local $arr1_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key))
           )
            (then
                (return (i32.load (i32.add(get_local $node)(i32.const 4))))
            )
       )
       (set_local $node (call $get_array (i32.load(i32.add (get_local $map) (i32.const 8))) (get_local $arr2_index)))
       (if (i32.and (i32.ne (get_local $node) (i32.const 0))
                    (i32.eq (i32.load (get_local $node))(get_local $key))
           )
            (then
                (return (i32.load (i32.add(get_local $node)(i32.const 4))))
            )
       )
       ;;Calling error func to raise key does not exist exception 
       (call $err)
       ;; Instead of returning -1, returning - MAXINT to indicate error
       (i32.const -2147483648)
    )
 
     ;; return the array length
    (func $len_map (param $map i32) (result i32)
        (i32.load (get_local $map))
    )
    (func $numOfRecords (param $map i32) (result i32)
        (i32.load (i32.add (get_local $map) (i32.const 12)))
    )"""